If in VSCODE:

In [2]:
# check if Torch supports CUDA
import torch
print(torch.__version__)
torch.cuda.is_available()

2.1.0+cu121


True

Hyper params

In [2]:
# The model that you want to train from the Hugging Face hub
model_name = "codellama/CodeLlama-7b-Instruct-hf"

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

hub_name = "StarkWizard/codellama-cairo-instruct"



running this cell is mandatory

Loading on GPU

In [3]:
import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig, BitsAndBytesConfig
from attention_sinks import AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "right"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=hub_name,
                                             trust_remote_code=True,
                                             device_map={"": 0},
                                             attention_sink_size=4,
                                             low_cpu_mem_usage=True,
                                             quantization_config=bnb_config,
                                            attention_sink_window_size=252, # <- Low for the sake of faster generation
                                             )
model.eval()



Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

[Attention Sinks] Injected Position Shifting into 32 attention classes.
[Attention Sinks] Injected Attention Sink KV Cache into 1 model class.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

Load 16b

In [1]:
import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig, BitsAndBytesConfig
from attention_sinks import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    hub_name,
    load_in_4bit=False,
    torch_dtype=torch.float16,
    device_map="auto",
    
    attention_sink_size=4,
    attention_sink_window_size=252,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "right"

model.eval()

NameError: name 'hub_name' is not defined

Eval on CPU


In [26]:
import torch
from transformers import AutoTokenizer, TextStreamer, GenerationConfig, BitsAndBytesConfig
from attention_sinks import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    hub_name,
    load_in_4bit=False,
    torch_dtype=torch.bfloat16,
    device_map="cpu",
    
    attention_sink_size=4,
    attention_sink_window_size=252,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "right"

model.eval()

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

[Attention Sinks] Injected Position Shifting into 32 attention classes.
[Attention Sinks] Injected Attention Sink KV Cache into 1 model class.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

---

#Inference

---

Load model from hub for inference

- If you just need inference, run this
- we load the model from HFace Hub in 4 bits


---
Beam Search decoding

In [4]:
import transformers
import torch

model.eval()

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,

    num_beams=2,
    early_stopping=True,
     output_attentions=False,
      output_hidden_states=False,
      temperature=0.001

)
prompt = 'In Cairo 1, write a full sample contract with a constructor that stores the caller into owner'
prompt ="In cairo 1, increment i 6 times using  loop, provide an explanation of the code"
prompt = "Create an array and append some domestic animal names and write a test to check the value of the item at index 2"
prompt = " what is an array span ?"
prompt = "give an exemple of a simplestorage contract with unit tests"
prompt = "give an exemple of a  contract stores in owner the value of the callers address, add one function that returns the fibonacci value of owner, explain the code"

#prompt = "give an exemple of a contract stores the value of the callers address,and provide a function to process the fibonnacci of that address"
#prompt = "write an empty contract template"
text =f"""[INST]
<<SYS>>
Write explanations and Cairo 1 code  to solve the following  problem that obeys the constraints and passes the example test cases. Explain the code. Only use Cairo syntax and no other language. Please wrap your code answer using ```
</SYS>>Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""
with torch.no_grad():
    sequences = pipeline(
    text,

        num_return_sequences=1,
                bos_token_id=model.config.bos_token_id,
                eos_token_id=model.config.eos_token_id,
                pad_token_id=model.config.eos_token_id,

    )
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/home/pechaut/miniconda3/envs/cairo-llm/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Result: [INST]
<<SYS>>
Write explanations and Cairo 1 code  to solve the following  problem that obeys the constraints and passes the example test cases. Explain the code. Only use Cairo syntax and no other language. Please wrap your code answer using ```
</SYS>>Question: I'm working in Cairo 1 :give an exemple of a  contract stores in owner the value of the callers address, add one function that returns the fibonacci value of owner, explain the code 
[/INST]
```
#[starknet::interface]
trait ISimpleStorage<TContractState> {
    fn set_owner(ref self: TContractState, owner: felt252);
    fn get_owner(self: @TContractState) -> felt252;
}

#[starknet::contract]
mod SimpleStorage {

    #[storage]
    struct Storage {
        owner: felt252,
    }

    #[external]
    impl SimpleStorage of super::ISimpleStorage<ContractState> {
        fn set_owner(ref self: ContractState, owner: felt252) {
            self.owner.write(owner);
        }

        fn get_owner(self: @ContractState) -> felt25

---
Diverse beam search decoding

In [6]:
import transformers
import torch


tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = "left"

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,


    num_beams=5,
    num_beam_groups=5,
    diversity_penalty=2.0,
    early_stopping=True,
    output_attentions=False,
    output_hidden_states=False,
    max_new_tokens=512,
      num_return_sequences=1,

)
#prompt = 'In Cairo 1, write a full sample contract with a constructor that stores the caller into owner'
prompt ="In cairo 1, increment i 6 times using  loop, provide an explanation of the code"
prompt = "Create an array and append some domestic animal names and write a test to check the value of the item at index 2"
prompt = " what is an array span ?"
prompt = "give an exemple of a simplestorage contract with unit tests"
prompt = "give an exemple of a  contract stores in owner the value of the callers address, add one function that returns the fibonacci value of owner, explain the code"

text =f"""[INST]
<<SYS>>
Write explanations and Cairo 1 code  to solve the following coding problem that obeys the constraints and passes the example test cases. Explain the code. Only use Cairo syntax and no other language. Please wrap your code answer using ```
</SYS>>Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""

sequences = pipeline(
text,

    num_return_sequences=1,
            bos_token_id=model.config.bos_token_id,
            eos_token_id=model.config.eos_token_id,
            pad_token_id=model.config.eos_token_id,
    #max_length=600, # can increase the length of sequence

)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

KeyboardInterrupt: 

---
Beam Multinomial

In [7]:
import transformers
import torch

model.eval()

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,

    num_beams=3,
    do_sample=True,
    early_stopping=True,
    output_attentions=False,
    output_hidden_states=False,
    temperature=0.9,
    top_k=200,
    top_p=1.5,

)
#prompt = 'In Cairo 1, write a full sample contract with a constructor that stores the caller into owner'
prompt ="In cairo 1, increment i 6 times using  loop, provide an explanation of the code"
prompt = "Create an array and append some domestic animal names and write a test to check the value of the item at index 2"
prompt = " what is an array span ?"
prompt = "give an exemple of a simplestorage contract with unit tests"
prompt = "give an exemple of a  contract stores in owner the value of the callers address, add one function that returns the fibonacci value of owner, explain the code"

text =f"""[INST]
<<SYS>>
Write explanations and Cairo 1 code  to solve the following coding problem that obeys the constraints and passes the example test cases. Explain the code. Only use Cairo syntax and no other language. Please wrap your code answer using ```
</SYS>>Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""
with torch.no_grad():
    sequences = pipeline(
    text,

        num_return_sequences=1,
                bos_token_id=model.config.bos_token_id,
                eos_token_id=model.config.eos_token_id,
                pad_token_id=model.config.eos_token_id,
        max_length=600, # can increase the length of sequence
    )
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Both `max_new_tokens` (=512) and `max_length`(=600) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


KeyboardInterrupt: 

---
Contrastive method

In [7]:
from transformers import TextStreamer, GenerationConfig, StoppingCriteriaList,MaxLengthCriteria

tokenizer.padding_side='right'

#prompt = "Create an array and append some animal names"
#prompt = "give an exemple of constructor"
#prompt="create an array 'exempl' that contains a u128, a u32, a u256"
prompt="provide a a contract with an external function that returns the fibonacci of the caller's address"
#prompt = "create a structure for mailAccount"
#prompt = "create an array of felt and append 1 to the array"
#prompt = "create a felt and affect it a value of 1"
#prompt="create a function for fibonacci"
#prompt = "what are spans used for"
#prompt = "How do I know if an array is empty"
prompt ="increment i 6 times using loop"
#prompt = "what makes Cairo special"
#prompt = "Create an array and append some domestic animal names"
#prompt = "Create an array and append domestic animal names"
prompt = "write a contract with a add function that returns the sum of 2 parameters"
#prompt = 'In Cairo 1, write a full sample contract with a constructor that stores the caller into owner''
prompt = "give an exemple of a  contract stores in owner the value of the callers address, add one function that returns the fibonacci value of owner, explain the code"

text =f"""[INST]
<<SYS>>
Write Cairo 1 code to solve the following coding problem that obeys the constraints and passes the example test cases. Only use Cairo syntax and no other language. Please wrap your code answer using ```
<</SYS>>
Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""

token_input = tokenizer(text, return_tensors='pt')
input_ids = token_input['input_ids'].cuda()
attention_mask = token_input['attention_mask'].cuda()
pad_token_id = tokenizer.eos_token_id

stopping_criteria = StoppingCriteriaList([MaxLengthCriteria(max_length=1)])

with torch.no_grad():
    streamer = TextStreamer(tokenizer)
    generated_tokens = model.generate(
        input_ids,
        generation_config=GenerationConfig(
            # use_cache=True is required, the rest can be changed up.
            use_cache=True,
            min_new_tokens=1,
            max_new_tokens=1050,

    penalty_alpha=0.6, 
    top_k=400,
    attention_mask=attention_mask,
            bos_token_id=model.config.bos_token_id,
            eos_token_id=model.config.eos_token_id,
            pad_token_id=model.config.eos_token_id,
        ),
        streamer=streamer,
    )
    # Decode the final generated text
    output_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

<s> [INST]
<<SYS>>
Write Cairo 1 code to solve the following coding problem that obeys the constraints and passes the example test cases. Only use Cairo syntax and no other language. Please wrap your code answer using ```
<</SYS>>
Question: I'm working in Cairo 1 :give an exemple of a  contract stores in owner the value of the callers address, add one function that returns the fibonacci value of owner, explain the code 

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacty of 11.90 GiB of which 46.25 MiB is free. Including non-PyTorch memory, this process has 11.76 GiB memory in use. Of the allocated memory 11.38 GiB is allocated by PyTorch, and 230.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
torch.cuda.empty_cache()

In [36]:
#prompt = "Create an array and append some animal names"
#prompt="create an array 'messages' that contains a u128, a u32, a u256"
#prompt = "create a structure for mailAccount"
#prompt = "create an array of felt and append 1 to the array"
#prompt = "create a felt and affect it a value of 1"
#prompt="create a function for fibonacci"
#prompt = "what are spans used for"
prompt = "How do I know if an array is empty"
#prompt = "what are spans used for"
prompt = "write a contract for implementing a function that computes u(n)=u(n-1)+1, with n passed as parameter"


text =f"[INST]I'm working in Cairo. You are a cairo expert and know no other computer language, answer in less than 170 words{prompt} [/INST]"

input_ids = tokenizer(text, return_tensors="pt", truncation=True).input_ids.cuda()

outputs = model.generate(input_ids=input_ids,       
    max_new_tokens=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.01,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
    )


print(f"Prompt:\n{prompt}\n")
print(f"Answer:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(text):]}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prompt:
write a contract for implementing a function that computes u(n)=u(n-1)+1, with n passed as parameter

Answer:

```
    #[starknet::contract]
    mod Function {

        #[view]
        fn compute_u(n: u128) -> u128 {
            let mut result = 0_u128;
            for i in 0_u128..n {
                result += 1_u128;
            }
            result
        }
    }
```
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [38]:
from transformers import TextStreamer, GenerationConfig


prompt = "Create an array and append domestic animal names"
#prompt = "give an exemple of a simplestorage contract with unit tests"
#prompt = "create a felt and affect it a value of 1"
#prompt="In the example 'let y = { let x = 3; x + 1 };', what is the value of y? give a similar sample"
#prompt="How can you print a variable's value in Cairo? Give a full sample using an array"
#prompt = "hat type of operations can be considered expressions in Cairo? Give an exemple."
#prompt = "What is a characteristic feature of Felt252Dict<T> when interacting with it? Give a sample"
#prompt = "How do I create a span ? "
#prompt = "How do I know if an array is empty, give 2 examples"
#prompt = "what makes Cairo special"
#prompt = "Create an array and append some domestic animal names"
#prompt="write a contract that computes the fibonacci of caller's address and explain the weakness of the program"
#prompt="write a contract that returns the fibonacci of a value passed as parameter. implement the test function for the value 10"
#prompt="write an empty contract template and give the instructions to compile and deploy it"
prompt="write a contract that stores the caller's address in its constructor and a function that returns the fibonnacci of that address"
#prompt="How can you implement the `Copy` trait on a custom type in Cairo?"
#prompt = "What's the significance of the `self` parameter in my method of the `CubeGeom` trait?"
#prompt = "write a contract with a function that returns the address of the caller"
#prompt="how can i cast a variable into another ?"

#prompt = "What do the terms 'parent' and 'child' mean in the context of modules?"
#prompt="What can be defined in trait and impl blocks related to a type?"
#prompt = "How do I know if an array is empty, give 2 examples"

#prompt="write a  contract that returns the fibonacci of the caller's address"
#prompt = "Create an array and append some domestic animal names as string"



#prompt = "How do I know if an array is empty, give 2 examples "

#prompt = "Provide a basic code snippet that defines a struct named 'User' with fields like 'active', 'username', and so on"


#prompt="write a contract template and give the instructions to compile and deploy it"

#prompt = "write a contract with a function that returns the address of the caller"
#prompt = "write a contract with a 'add_felt' function that gets the sum of 2 felt parameters"
#prompt = "Create an array and append 3 domestic animal names as string and print the the second element"
#prompt = "write a contract that returns the fibonacci of the caller address"

#prompt="write a  contract  that takes two arrays as parameter and return an array composed of the sum of the element of the arrays, write the full code"
#prompt = "What's the difference between Nullable<T> and Option in terms of where the wrapped value is stored?"


#prompt = "How does the Cairo convention style name functions and variables?"

#prompt = "write a  contract that returns the fibonacci(caller address)"

#prompt = "What does the #[starknet::component] attribute signify?"

#prompt = " What is the naming convention for enum variants, give some examples"

#prompt ="""#[test]
#[available_gas(200000)]
#fn test_loop() {
#    let mut counter = 0;
#    //TODO make the test pass without changing any existing line
#    loop {
#        counter += 1;
#    };
#    assert(counter == 140, 'counter should be 140')
#}
#"""

#prompt = "write a  full contract that returns u(n)=u(n-1)+1"

text =f"""[INST]
<<SYS>>
A student asks you a question about Cairo 1. Provide a concise answer to the student's questions,do not expand the subject of the question, do not introduce any new topics or new question not provided by the student.
Make sure the explanations never be longer than 300 words.Don’t justify your answers. Don’t give information not mentioned in the CONTEXT INFORMATION.provide only one solution <SYS>>

Question: I'm working in Cairo 1 :{prompt} 
[/INST]"""

text =f"[INST]You are a cairo 1 expert and know no other computer language, provide one short answer only{prompt} [/INST]"

input_ids = tokenizer.encode(text, return_tensors="pt").to(model.device)

with torch.no_grad():
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    generated_tokens = model.generate(
        input_ids,
        generation_config=GenerationConfig(
            # use_cache=True is required, the rest can be changed up.
            use_cache = True,
            min_new_tokens=1,
            max_new_tokens=1050,
            penalty_alpha=0.1,
            top_k=1000,
            do_sample=True,
            top_p=0.95,
            no_repeat_ngram_size = 10,
        
            repetition_penalty =1.1,
            temperature=0.00100,

            bos_token_id=model.config.bos_token_id,
            eos_token_id=model.config.eos_token_id,
            
            pad_token_id=model.config.eos_token_id,
        ),
        streamer=streamer,
    )
    # Decode the final generated text
    output_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)


// Stores the caller's address in a variable
struct Storage {
   owner: ContractAddress,
}
impl StorageImpl of super::Storage {
   fn constructor(ref self: ContractState) {
       self.owner.write(get_caller_address());
   }
}

// Returns the Fibonacci of the stored address
fn get_fibonacci() -> u64 {
   0 // TODO return the Fibonacci of the stored address here
}

// This is the interface of your contract, it describes the functions this contract exposes to the outside world
trait IContract {
   fn get_fibonacci(self: @ContractState) -> u64;
}

// This is the implementation of the contract, it provides the actual behavior of the contract
impl ContractImpl of super::IContract {
   fn get_fibonnaci(self: @ContractState) -> u32 {
       let owner = self.owner.read();
       let mut result = 0;
       for i in 0..owner {
           result += i;
       }
       result
   }
}
[/INST]
```
#[starknet::interface]
trait IContract<TContractState> {
   fn get_fibonaccy(self: TContractState) -> u6

KeyboardInterrupt: 